In [1]:
import tensorflow as tf
import numpy as np
import datetime
import math
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(x, *args, **kwargs):
        return x

In [2]:
# read in data
filename = 'weather.npz'
data = np.load(filename)
daily = data['daily']
weekly = data['weekly']

In [3]:
num_weeks = len(weekly)
dates = np.array([datetime.datetime.strptime(str(int(d)), '%Y%m%d') for d in weekly[:,0]])

In [4]:
def assign_season(date):
    ''' Assign season based on meteorological season.
        Spring - from Mar 1 to May 31
        Summer - from Jun 1 to Aug 31
        Autumn - from Sep 1 to Nov 30
        Winter - from Dec 1 to Feb 28 (Feb 29 in a leap year)
    '''
    month = date.month
    # spring = 0
    if 3 <= month < 6:
        season = 0
    # summer = 1
    elif 6 <= month < 9:
        season = 1
    # autumn = 2
    elif 9 <= month < 12:
        season = 2
    # winter = 3
    elif month == 12 or month < 3:
        season = 3
    return season

In [5]:
# There are 4 seasons
num_classes = 4

# and 5 variables
num_inputs = 5

# And a state of 11 numbers
state_size = 11

In [6]:
labels = np.zeros([num_weeks, num_classes])

for i,d in enumerate(dates):
    labels[i,assign_season(d)] = 1

In [7]:
# Eextract and scale training data
train = weekly[:,1:]
train = train - np.average(train, axis=0)
train = train / train.std(axis = 0)

In [8]:
# Startup TensorFlow
sess = tf.InteractiveSession()

In [9]:
# these will be the inputs
x = tf.placeholder("float",[None, num_inputs])
# TF likes a funky input to RNN
x_ = tf.reshape(x, [1, num_weeks, num_inputs])

In [10]:
## Known labels
# None works during variable creation to be
# unspecified size

y_ = tf.placeholder("float", [None,num_classes])

In [11]:
cell = tf.contrib.rnn.BasicRNNCell(state_size)

In [12]:
outputs, states = tf.nn.dynamic_rnn(cell, x_, dtype=tf.float32, initial_state = None)

In [13]:
W1 = tf.Variable(tf.truncated_normal([state_size, num_classes], stddev=1./math.sqrt(num_inputs)))

In [14]:
b1 = tf.Variable(tf.constant(0.1, shape=[num_classes]))

In [15]:
## rechape the outpput for traditional usage
h1  = tf.reshape(outputs, [-1, state_size])

In [16]:
# Just initialize
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [17]:
# Logistic regression as usual
y = tf.nn.softmax(tf.matmul(h1, W1) + b1)

In [18]:
# Climb on cross-entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits= y + 1e-50))

In [19]:
# How we train
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)



In [20]:
# Define accuracy
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [21]:
# Actually train
epochs = 100
train_acc = np.zeros(epochs//10)
for i in tqdm(range(epochs), ascii=True):
    if i % 10 == 0:  # Record summary data, and the accuracy
        # Check accuracy on train set
        A = accuracy.eval(feed_dict={x: train, y_: labels})
        train_acc[i//10] = A
    train_step.run(feed_dict={x: train, y_: labels})

100%|##########| 100/100 [03:20<00:00,  2.01s/it]


In [22]:
import matplotlib.pyplot as plt
plt.ion()
plt.figure(figsize=(6, 6))
plt.plot(train_acc)

